# Circle Segment Intersection (for interpolation)
Here is an interactive plot that demonstrates the functionality of the formula to calculate the intersection of a line segment, and a circle centered at the origin.

In [8]:
from bqplot import *
import numpy as np
import ipywidgets as widgets


def circle_segment_intersection(p1, p2, r):
    x1, y1 = p1
    x2, y2 = p2
    dx = x2 - x1
    dy = y2 - y1
    dr2 = dx ** 2 + dy ** 2
    D = x1 * y2 - x2 * y1
    d1 = x1 ** 2 + y1 ** 2
    d2 = x2 ** 2 + y2 ** 2
    dd = d2 - d1
    sqrt_term = np.sqrt(r ** 2 * dr2 - D ** 2)
    x = (D * dy + np.copysign(1.0, dd) * dx * sqrt_term) / dr2
    y = (-D * dx + np.copysign(1.0, dd) * dy * sqrt_term) / dr2
    return x, y


MAX = 5.0
x_sc = LinearScale(min=-MAX, max=MAX)
y_sc = LinearScale(min=-MAX, max=MAX)

ax_x = Axis(label="x", scale=x_sc, tick_format="0.0f")
ax_y = Axis(label="y", scale=y_sc, orientation="vertical", tick_format="0.0f")

points = Scatter(
    names=["A", "B"], x=[0.0, 3.0], y=[2.0, 4.0], scales={"x": x_sc, "y": y_sc}, enable_move=True
)


def get_circle(r):
    t = np.linspace(0, 2 * np.pi)
    x = r * np.cos(t)
    y = r * np.sin(t)
    return x, y

radius_slider = widgets.FloatSlider(min=0.0, max=MAX, value=3.0, description="Circle radius")
circle_x, circle_y = get_circle(radius_slider.value)

circle = Lines(x=circle_x, y=circle_y, scales={"x": x_sc, "y": y_sc}, colors=["green"])

x1, x2 = points.x
y1, y2 = points.y
xi, yi = circle_segment_intersection((x1, y1), (x2, y2), radius_slider.value)

intersection = Scatter(
    names=["C"],
    x=[xi],
    y=[yi],
    scales={"x": x_sc, "y": y_sc},
    enable_move=False,
    colors=["purple"],
)

fig = Figure(axes=[ax_x, ax_y], marks=[circle, points, intersection])

fig.max_aspect_ratio = 1
fig.min_aspect_ratio = 1


def both_inside_or_both_outside_circle(points, r):
    x1, x2 = points.x
    y1, y2 = points.y
    d1 = x1 ** 2 + y1 ** 2
    d2 = x2 ** 2 + y2 ** 2
    if d1 < r ** 2 and d2 < r ** 2:
        return True
    elif d1 > r ** 2 and d2 > r ** 2:
        return True
    else:
        return False


def update_circle(message):
    circle_x, circle_y = get_circle(radius_slider.value)
    circle.x = circle_x
    circle.y = circle_y
    update_intersection(message)


def update_intersection(message):
    x1, x2 = points.x
    y1, y2 = points.y
    r = radius_slider.value
    if both_inside_or_both_outside_circle(points, r):
        circle.colors = ["red"]
        intersection.x = []
        intersection.y = []
    else:
        circle.colors = ["green"]
        xi, yi = circle_segment_intersection((x1, y1), (x2, y2), r)
        intersection.x = [xi]
        intersection.y = [yi]


points.observe(update_intersection, ["x", "y"])

radius_slider.observe(update_circle, "value")

widgets.VBox(
    [
        widgets.Label(
            "A and B can be moved with the mouse. One must be inside the circle, and one must be outside.",
            fixed=True,
        ),
        radius_slider,
        fig,
    ]
)